## 업무 자동화 - 웹 크롤링

주피터로 정리하는게 가독성 측면에서 나을 것 같아서 새로 만듬. <br>
가능하면 큰 묶음 별로 cell을 나눠서 **cell 별로 실행 가능**하게 해주고 그에 맞는 설명 달면서 작성해주자. <br>
***기본적으로 조건이 걸리는 부분은 하나의 cell로 해주어야 함.*** (*아래에 쪼갠건 수정할 때 다시 합치셈.*) <br>
예시는 아래에 import 부분만 작성함.

#### 1) 활용 패키지 import 
selenium : 웹 크롤링 <br>
beautifulsoup : html 출력 포맷 변환 <br>
pandas : dataframe 형식 사용 <br>
re : 정규식 사용 <br> 
datetime : 시간 형식 사용 <br>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  8 21:03:36 2017

@author: HS
"""

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import datetime

In [ ]:
# Chrome driver를 실행시킨다. 
driver = wb.Chrome('C:/Users/HS/Desktop/과외/studyroom/chromedriver.exe')
driver.implicitly_wait(2)

# 날짜 관련 변수 지정
today = datetime.date.today()
week_ago = today - datetime.timedelta(7)
week_day_ago = today - datetime.timedelta(8)
today = today.strftime('%Y/%m/%d')
week_ago = week_ago.strftime('%Y/%m/%d')
week_day_ago = week_day_ago.strftime('%Y/%m/%d')

#조달청계약요청
driver.get("http://www.g2b.go.kr:8091/cm/contstus/fwdPpsItemContractReqStus.do")
driver.find_element_by_xpath('//*[@id="prodNm"]').send_keys('전반')
driver.find_element_by_xpath('//*[@id="srchFrm"]/div[2]/div/a[1]/span').click()

## 최종적으로 확인할 리스트들 지정
date_sum = []
money_sum = []
number_sum = []
region_sum = []

##지역 패턴을 만들어 준다
pattern = re.compile(r'대전|충청|충북|충남|세종')
### re.search(pattern, 지역명) # 해당 지역이 있으면 True, 없으면 False로 뜰 것이다.

## 총 5개의 페이지(최대)로 설정
pages = 5

<font color='blue'>
### 참고

위에서 re 정규식에 대해서 설명하도록 하겠다.
```python 
pattern = re.compile(r'대전|충청|충북|충남|세종')
```
먼저, pattern을 지정해주는데 이는 compile이라는 함수를 사용한다. <br>
그리고 이 pattern을 사용해서 내가 원하는 문자열 내에 해당 문자가 존재하는지 봐야 하는데 이 때는 
search 함수를 사용한다. <br>
예시 및 사용 방법은 아래와 같다. 이를 참조해서 re패키지를 써서 조건을 걸어보도록 하자.
```python
city = '대전광역시'
if re.search(pattern, city):
    print('도시 이름이 원하는 조건에 부합한다.')
```

In [ ]:
## 최대 5개의 페이로 지정한 부분을 for문으로 돌림. 전체 페이지를 돌리지 않고, 일정 조건에 부합하면 break 할 예정
for i in range(pages):
    
    ## 소스를 가져와 soup으로 정리
    html = driver.page_source
    soup = bs(html, 'lxml')
    
    
    ## '클릭' 포함하여 총 6개의 원하는 '일회성' 정보를 가져옴
    ## 추가적으로 date 라는 리스트를 생성하여, 원하는 날짜까지의 정보만 필터
    ## ex) date_tem이 30개라도 date는 원하는 날짜에 멈추므로 17개가 되면, 그 17개에 한해서
    ## 2억 또는 그 외 다른 조건들을 탐색
    date_tem = [j.text.strip() for j in soup.find_all(attrs = {'class':'tc'}) if '/' in j.text]
    money = [j.text.strip() for j in soup.find_all(attrs = {'class':'tr'}) if '원' in j.text]
    money = [int(i.replace('원','').replace(',','')) for i in money]
    number = [j.text.strip() for j in soup.find_all(name = 'a', attrs = {'href' : '#'})][1:]
    region = [j.text.strip() for j in soup.find_all(attrs = {'class':'tl'})][1::2]
    click = [j.text.strip() for j in soup.find_all(attrs = {'class':'default'}) if '더보기' in j.text]
    date = [] 
    
    ## for 문을 통해서 1페이지씩 멈추고 싶은 날짜가 있는지 확인
    ## 확인하여, 원하는 날짜까지만의 정보를 다시 가져올 예정
    ## 기존의 date를 date_tem으로 바꾸고, 다시 date라는 List 생성
    ## date라는 리스트에 원하는 조건까지만 추가!
#==============================================================================
#     for k in range(len(date_tem)):
#         
#         if date_tem[k] == '2017/07/19':
#             break
#         date.append(date_tem[k])
#==============================================================================
    
    ## 조건을 하나로 합쳐본다 : 날짜가 원하는 날짜가 안나오고, 금액이 1억이 넘으면 클릭
    ## 클릭해서 원하는 지역의 조건이 맞는지 확인해본다(매번 클릭할 때마다 html 새로 지정)
    for k in range(len(date_tem)):
        
        if date_tem[k] != '2017/07/19' and money[k] >= 1e8:

<font color='blue'>
생각해보니 날짜가 전 날짜가 있을 수도 있어서 그럼 이 로직에 안 걸리면서도 for 문이 돌아갈 수 있음. 부등호로 하는게 유리할 듯함. <br>
그리고 python에선 else if --> elif로 사용함.  <br>
위에서 if문 쓰고 아래에서 elif를 사용하였는데 그거보다는 if money로 한 번 거르고 아래에서 if로 date조건으로 끊어주는게(break) 어떨까 함. <br>
if date_tem[k] != '2017/07/19' 이 내용이 별 다른 조건이 없는 듯함. (어차피 내림차순이라)     

그리고 아래 코드에서 
```python
if '대전'or'충청'or'충북'or'충남'or'세종' in soup.find(attrs = {'class':'tl'}).text.strip():

else if '대전'or'충청'or'충북'or'충남'or'세종' in soup.find(attrs = {'colspan':'5'}).text.strip():
```
조건 자체가 같으므로 if를 한 번만 쓰고 확인할 내용을 OR로 묶어주는게 어떨까 싶음

아래 코드에선 
```python
if click != []:
    driver.find_element_by_link_text(click[0]).click()
else:
    break
````
click != [] 이 아무것도 없다는 의미를 나타내는 듯한데 에러가 날듯 함 아무것도 없다는 표현은 주로 None으로 나타냄. <br>
웹에서 관련 표현 찾아보자. 

In [ ]:
            driver.find_element_by_link_text(number[k]).click()
            html = driver.page_source
            soup = bs(html, 'lxml')
            
            ## 첫번째로 '납품장소'를 확인한다
            ## 지역이름이 들어갔는지 확인한다
            ## 우선 if문으로 re 함수를 쓸 수 있는 방법이 안떠올라, 각각 or로 입력한다.
            ### 위에서 re.search
            
            ## 'class'가 'tl'일 때 가장 첫번 째 나오는 값이 '납품장소'의 값이므로 아래와 같이 입력한다
            if '대전'or'충청'or'충북'or'충남'or'세종' in soup.find(attrs = {'class':'tl'}).text.strip():
                
            ## 그다음 두번째로는 '공사명'을 확인한다
            ## 이 또한 지역이름이 들어갔는지 확인한다
            ## '공사명'의 값은 colspan:5 인 유일한 값이므로 아래와 같이 입력한다
            else if '대전'or'충청'or'충북'or'충남'or'세종' in soup.find(attrs = {'colspan':'5'}).text.strip():
            
            ## 마지막으로 '발주(공고)기관'의 전화번호를 확인한다
            
            else if 
                
                
        else if date_tem[k] == '2017/07/19':
            break
    
    ## '클릭(+더보기)' 버튼을 누름
    ## 단, 클릭 버튼이 없으면 error가 뜨기 때문에(page 수를 최대 5로 지정했으나, 그보다 적을 경우)
    ## 클릭 버튼이 없을 경우 break를 할 수 있는 조건문 생성
    if click != []:
        driver.find_element_by_link_text(click[0]).click()
    else:
        break


            
'''
    우선 if문으로 지정한 날짜가 맞으면 멈춤!
    if date[j] == week_day_ago:
        break
    
    금액이 2억이 넘으면 클릭한다!
    저번 숙제에 쓴대로 우선 2억이 넘으면 모두 클릭할 예정
    if money[j] == 2e8:
        driver.find_element_by_link_text(number[j]).click()
        
        3가지 순서대로 지역에 맞는지 확인
        납품장소 - 공사명 - 발주(공고)기관
        또한, 각 공고 내의 위 사항들은 모두 같은 xpath를 가짐을 확인
        
        if 납품장소 == "대전~~~":
            필요한 항목들을 끌고옴
            
        else if 공사명 == "대전~~~~":
            필요한 항목들을 끌고옴
            
        else if 발주(공고)기관 == "041", "042", "043", "044":
            필요한 항목들을 끌고옴
            
        else if 발주(공고)기관을 '네이버지도'에 검색하여 상위 3개를 확인하여 "대전~~~"이 있으면:
            필요한 항목들을 끌고옴
            
        else:
            2억 이상의 리스트를 확인할 수 있도록 엑셀로 만들어서 확인
            이건 그때 그때 마다 보고 지울 생각.
            또한, 원하는 지역이 아니었던 발주(공고)기관들은 리스트로 만들어서
            추후에 네이버 지도에 검색하는 것보다 우선적으로 리스트를 확인할 수 있도록 만들고 싶음
    
    내가 원하는 결론은
    1) 필요한 항목들을 엑셀로 만들어서 보는 것
    2) 내가 걸러낸 항목(2억 이하)에 대해서 맞는지 확인
       -> 그래서 납품장소, 공사명도 가져와야 되는지 의문..
    3) 내가 걸러낸 발주(공고)기관들은 계속 데이터로 가지고 있을 예정 '''


                    